# アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


ブレンディング
バギング
スタッキング

## 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。


## scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

In [155]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [156]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [157]:
y = data.loc[:, 'SalePrice']
X = data.loc[:, ['GrLivArea', 'YearBuilt']]

In [158]:
y = y.values
X = X.values

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# ブレンディング

## 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


## ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation


In [21]:
class Blending():
    def __init__(self, models, weights=None):
        self.models = models
        self.weights = weights
        
    def fit(self, X, y):
        self.models_ = []
        for mdl in self.models:
            fitted_mdl = mdl.fit(X, y)
            self.models_.append(fitted_mdl)
        return self
    
    def predict(self, X):
        pred = np.asarray([mdl.predict(X) for mdl in self.models_])
        avg_pred = np.average(pred, axis=0, weights=self.weights)
        return avg_pred

上から順に
- ロジスティック回帰
- 決定木
- SVM  
で分類した際の平均二乗誤差（MSE）の値を算出

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import mean_squared_error

clf1 = LinearRegression()
clf2 = DecisionTreeClassifier()
clf3 = SVC()

for clf in [clf1, clf2, clf3]:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = mean_squared_error(y_true=y_test, y_pred=y_pred)
    print(score)

2016350238.211333
2590490870.1061645
4980345342.777397


複数の分類器をブレンディングさせる。   
分類器(重み)
- ロジスティック回帰(0.6)
- 決定木(0.2)
- SVM(0.2)

In [23]:
mv_clf = Blending(models=[clf1, clf2, clf3], weights=[0.6, 0.2, 0.2])
mv_clf.fit(X_train, y_train)
y_pred_mv = mv_clf.predict(X_test)
score = mean_squared_error(y_true=y_test, y_pred=y_pred_mv)
print(score)

1874264303.6311107


- ロジスティック回帰(0.7)
- 決定木(0.3)

In [24]:
mv_clf2 = Blending(models=[clf1, clf2], weights=[0.7, 0.3])
mv_clf2.fit(X_train, y_train)
y_pred_mv2 = mv_clf2.predict(X_test)
score2 = mean_squared_error(y_true=y_test, y_pred=y_pred_mv2)
print(score2)

1817367571.462801


- ロジスティック回帰(0.9)
- SVM(0.1)

In [25]:
mv_clf3 = Blending(models=[clf1, clf3], weights=[0.9, 0.1])
mv_clf3.fit(X_train, y_train)
y_pred_mv3 = mv_clf3.predict(X_test)
score3 = mean_squared_error(y_true=y_test, y_pred=y_pred_mv3)
print(score3)

2042899084.3169372


ロジスティック回帰の重みをかなり大きくしたが、ロジスティック回帰単体よりMSEは少し大きくなった。

# バギング

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


## バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [11]:
class Bagging():
    def __init__(self, base_estimator, n_estimators):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        
    def fit(self, X, y):
        self.models_ = []
        #X_slct, _, y_slct, _ = train_test_split(X, y, shuffle=True)
        random_index = np.random.choice(np.arange(X.shape[0]), size=X.shape[0])
        X_slct = X[random_index]
        y_slct = y[random_index]
        for i in range(self.n_estimators):
            fitted_mdl = self.base_estimator.fit(X_slct, y_slct)
            self.models_.append(fitted_mdl)
        return self
    
    def predict(self, X):
        pred = np.asarray([mdl.predict(X) for mdl in self.models_])
        avg_pred = np.average(pred, axis=0)
        return avg_pred


決定木のバギング

In [12]:
tree = DecisionTreeClassifier()

bg = Bagging(base_estimator=tree, n_estimators=10)
bg.fit(X_train, y_train)
y_pred_bg = bg.predict(X_test)
score = mean_squared_error(y_true=y_test, y_pred=y_pred_bg)
print(score)

2291112180.8630137


In [13]:
bg = Bagging(base_estimator=tree, n_estimators=30)
bg.fit(X_train, y_train)
y_pred_bg = bg.predict(X_test)
score = mean_squared_error(y_true=y_test, y_pred=y_pred_bg)
print(score)

2235680785.712329


MSEが下がっていることが分かる

# スタッキング

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


## スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずはK0=3,M0=2程度にします。

#### 《学習時》


（ステージ0）


- 学習データをK0個に分割する。
- 分割した内の(K0−1)個をまとめて学習用データ、残り1個を推定用データとする組み合わせがK0個作れる。
- あるモデルのインスタンスをK0個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り1個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスもK0個用意し、同様のことを行う。モデルがM0個あれば、M0個のブレンドデータが得られる。

（ステージn）
- ステージn−1のブレンドデータをMn−1次元の特徴量を持つ学習用データと考え、Kn個に分割する。以下同様である。

（ステージN）＊最後のステージ

- ステージN−1のMN−1個のブレンドデータをMN−1次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

#### 《推定時》


（ステージ0）

- テストデータをK0×M0個の学習済みモデルに入力し、K0×M0個の推定値を得る。これをK0の軸で平均値を求め M0次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージn）

- ステージn−1で得たブレンドテストをKn×Mn個の学習済みモデルに入力し、Kn×Mn個の推定値を得る。これをKnの軸で平均値を求めM0次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージN）＊最後のステージ


- ステージN−1で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [164]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

In [169]:
class Stacking():
    def __init__(self, models, meta_model, k0, n_stage):
        self.models = models
        self.meta_model = meta_model
        self.k0 = k0
        self.n_stage = n_stage
        
    def train(self, X, y):

        self.models_ = []

        kf = KFold(n_splits=k0)
        blend_data = np.empty((int(len(X)*1/self.k0), 0))

        for mdl in models: 
            blend_data_val = np.empty((int(len(X)*1/self.k0), ))

            for train_idx, test_idx in kf.split(X, y):
                fitted_mdl = mdl.fit(X[train_idx], y[train_idx])
                models_.append(fitted_mdl)

                blend_data_val += fitted_mdl.predict(X[test_idx])

            blend_data = np.append(blend_data, (blend_data_val / self.k0)[:, np.newaxis], axis=1)
        
        blend_data_val = lab_enc.fit_transform(blend_data_val)
        return blend_data, blend_data_val

    def last_train(self, X, y):
        self.meta_model.fit(X, y)

        return self
        
    def fit(self, X, y):
        self.models_set = []
        blend_data, blend_data_val = self.train(X, y)
        for i in range(self.n_stage):

            blend_data, blend_data_val = self.train(blend_data, blend_data_val)
            
        self.last_train(blend_data, blend_data_val)
        
        return self        
            
    def _predict(self, X, models_set):
        pred = np.asarray([mdl.predict(X) for mdl in models_set])
        avg_pred = np.average(pred, axis=0)
        return avg_pred
        
    def last_predict(self, X):
        return self.meta_model.predict(X)
        
    def predict(self, X):
        blend_test_set = self._predict(X, self.models_set[0])
        for i in range(self.n_stage):
            blend_test_set = self._predict(X, self.models_set[i+1])
            
        return self.last_predict(blend_test_set)
        

In [170]:
clf1 = LinearRegression()
clf2 = DecisionTreeClassifier()
clf3 = SVC()

for clf in [clf1, clf2, clf3]:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = mean_squared_error(y_true=y_test, y_pred=y_pred)
    print(score)

1772803242.9021819
2834091131.191781
4345032720.239726


- ステージ数:3
- 分類器: ロジスティック回帰, SVC, 決定木(最後のステージ)
- データ分割数: 4

In [177]:
stck = Stacking(models=[clf1, clf3], meta_model=clf2, k0=4, n_stage=1)
stck.fit(X_train, y_train)
y_pred = mv_clf.predict(X_test)
score = mean_squared_error(y_true=y_test, y_pred=y_pred)
print(score)

1399606044.9352725


MSEは単一モデルより低くなったことが確認できる。